In [1]:
from pymongo import MongoClient
import json
from tqdm import tqdm

Load data into MongoDB

In [10]:
from pathlib import Path
import os
import json


# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['RAG']
collection = db['PubMedEmbedded']
# Define the path for the source directory
source_directory = Path('C:/Users/linus/big_data/pubmed/test_chunk_embedded/')

# Check if the source directory exists
if not source_directory.exists():
    print("The source directory does not exist.")
else:
    # Iterate through each file in the source directory
    for file_name in tqdm(os.listdir(source_directory)):
        if file_name.endswith('.jsonl'):
            source_file = source_directory / file_name
            
            # Open and read the JSONL file containing the PubMed documents
            with open(source_file, 'r') as json_file:
                for line in json_file:
                    data = json.loads(line)
                    collection.insert_one(data)

100%|██████████| 10/10 [02:01<00:00, 12.19s/it]


In [8]:
from pathlib import Path
import os
import json

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['RAG']
collection = db['PubMed']
# Define the path for the source directory
source_directory = Path('C:/Users/linus/big_data/pubmed/test_chunk_embedded/')


def search_TF_IDF(query):
    results = collection.find({"$text": {"$search": query}}).limit(100)
    return results

pmid_liste = search_TF_IDF("List signaling molecules (ligands) that interact with the receptor EGFR?")
for i in pmid_liste:
    print(i['PMID'])

1309297
1389099
1880949
2458073
1851518
3015394
2576295
1370492
1314448
2926839
1845982
1716463
2810532
1447576
1832907
1383230
1538276
1369801
1522533
1678600
2755700
1772304
2553748
1320909
1717146
2464868
2884496
1459725
2849039
2106751
2214281
2154210
2572319
2083199
1445750
1730638
2645219
1347529
2065272
2396042
2690971
2482148
2372767
1973074
1357997
2915986
2367059
1349015
1391697
2022478
2848004
1394216
2021959
1375794
2004618
2031413
1584765
1503407
2014751
2380256
2162688
2285594
1980602
1360720
2009534
2675297
1718587
2626045
1419645
1457340
2025065
1846551
2253244
2803944
1739615
1484435
1808202
2162976
1728365
1751401
1696918
1627426
1873798
2024456
1672440
2626279
2624856
1904421
2161199
1320666
2790793
2463445
2701167
2570489
2606569
2659165
2311071
2508310
1867098
1503912


In [2]:
from Embedding import TextEmbedder
embedder = TextEmbedder()

In [28]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

collection2 = db['PubMedEmbedded']


def find_nearest_embeddings(query_embedding, embeddings, n=10):
    # Initialisieren Sie NearestNeighbors
    nearest_neighbors = NearestNeighbors(n_neighbors=n, algorithm='auto').fit(embeddings)
    # Finden Sie die n nächsten Nachbarn des Suchtext-Embeddings
    distances, indices = nearest_neighbors.kneighbors([query_embedding])
    return indices[0], distances[0]

def search_and_retrieve_embeddings(query):
    # Textsuche durchführen und gleichzeitig die Embeddings abrufen
    results = collection2.find(
        {"$text": {"$search": query}},
        {'PMID': 1, 'embeddings': 1, '_id': 0}
    ).limit(1000)
    
    # Extrahieren Sie PMIDs und Embeddings aus den Ergebnissen
    pmids = []
    embeddings = []
    for doc in results:
        pmids.append(doc['PMID'])
        embeddings.append(doc['embeddings'])

    # Konvertieren Sie die Embeddings-Liste in ein NumPy-Array
    embeddings_array = np.array(embeddings)

    return pmids, embeddings_array

# Suchtext-Embedding generieren
such_text = "List signaling molecules (ligands) that interact with the receptor EGFR?"
pmids, embedding_matrix = search_and_retrieve_embeddings(such_text)
suchtext_embedding = embedder.embed(such_text)

# Finde die nächsten 10 Embeddings
indices, distances = find_nearest_embeddings(suchtext_embedding, embedding_matrix)

# Ergebnisse anzeigen
for i, index in enumerate(indices):
    print(f"{i + 1}: PMID = {pmids[index]}, Distanz = {distances[i]}")

1: PMID = 219769, Distanz = 4.967927935519385
2: PMID = 1813, Distanz = 4.968760284334843
3: PMID = 233131, Distanz = 4.979666667735479
4: PMID = 200939, Distanz = 4.981703585378111
5: PMID = 168935, Distanz = 4.984101335649784
6: PMID = 268625, Distanz = 4.987606585279695
7: PMID = 189643, Distanz = 5.0054406655705765
8: PMID = 200712, Distanz = 5.01118222034559
9: PMID = 95806, Distanz = 5.036777312349309
10: PMID = 175945, Distanz = 5.0384221205776685
